In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from hyperopt import STATUS_OK, Trials, hp, space_eval, tpe, fmin
import lightgbm as lgb
from sklearn.ensemble import RandomForestClassifier
from matplotlib import pyplot as plt
from matplotlib import rcParams
import numpy as np
from pathlib import Path
import pandas as pd
from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedKFold, train_test_split
import seaborn as sns
import warnings
from sklearn.linear_model import LogisticRegression
import numpy as np


In [3]:
rcParams['figure.figsize'] = (16, 8)
plt.style.use('fivethirtyeight')
pd.set_option('max_columns', 100)
pd.set_option("display.precision", 4)
warnings.simplefilter('ignore')

In [4]:
data_dir = Path('input')
feature_dir = Path('feature')
val_dir = Path('val')
tst_dir = Path('tst')
sub_dir = Path('sub')

trn_file = data_dir / 'train.csv'
tst_file = data_dir / 'test.csv'
sample_file = data_dir / 'sample_submission.csv'

target_col = 'class'
n_fold = 5
n_class = 3
seed = 42

In [5]:
algo_name = 'lgbcv'
feature_name = 'stacking1_final'
model_name = f'{algo_name}_{feature_name}'

feature_file = feature_dir / f'{feature_name}.csv'
p_val_file = val_dir / f'{model_name}.val.csv'
p_tst_file = tst_dir / f'{model_name}.tst.csv'
sub_file = sub_dir / f'{model_name}.csv'

In [6]:
model_names = ['lrcv_polyfeature_최종', 'RF_hyperopt_feature_final', 'lgbcv_feature_final','NN_hyperopt_feature_final']
trn = []
tst = []
feature_names = []
for model in model_names:
    trn.append(np.loadtxt(val_dir / f'{model}.val.csv', delimiter=','))
    tst.append(np.loadtxt(tst_dir / f'{model}.tst.csv', delimiter=','))
    feature_names += [f'{model}_class0', f'{model}_class1', f'{model}_class2']
    
trn = np.hstack(trn)
tst = np.hstack(tst)
feature_names

['lrcv_polyfeature_최종_class0',
 'lrcv_polyfeature_최종_class1',
 'lrcv_polyfeature_최종_class2',
 'RF_hyperopt_feature_final_class0',
 'RF_hyperopt_feature_final_class1',
 'RF_hyperopt_feature_final_class2',
 'lgbcv_feature_final_class0',
 'lgbcv_feature_final_class1',
 'lgbcv_feature_final_class2',
 'NN_hyperopt_feature_final_class0',
 'NN_hyperopt_feature_final_class1',
 'NN_hyperopt_feature_final_class2']

In [7]:
y = pd.read_csv(trn_file, index_col=0, usecols=['id', target_col]).values.flatten()

In [8]:
X_trn, X_val, y_trn, y_val = train_test_split(trn, y, test_size=.2, random_state=seed)

In [9]:
params = {
    'objective': 'multiclass', 'n_estimators': 1000, 'subsample_freq': 1, 'random_state': 42, 'n_jobs': -1, 'bagging_fraction': 0.5, 'colsample_bytree': 0.8, 'learning_rate': 0.013114188498150362, 'min_child_samples': 50, 'num_leaves': 15, 'subsample': 0.7000000000000001
}


def objective(hyperparams):
    model = lgb.LGBMClassifier(**params, **hyperparams)
    model.fit(X=X_trn, y=y_trn,
              eval_set=[(X_val, y_val)],
              eval_metric="multi_logloss",
              early_stopping_rounds=10,
              verbose=False)
    score = model.best_score_["valid_0"]["multi_logloss"]

    return {'loss': score, 'status': STATUS_OK, 'model': model}

trials = Trials()
best = fmin(fn=objective, space=space, trials=trials,
            algo=tpe.suggest, max_evals=10, verbose=1)

hyperparams = space_eval(space, best)
n_best = trials.best_trial['result']['model'].best_iteration_


In [10]:
cv = StratifiedKFold(n_splits=n_fold, shuffle=True, random_state=seed)

In [11]:
p_val = np.zeros((trn.shape[0], n_class))
p_tst = np.zeros((tst.shape[0], n_class))
for i, (i_trn, i_val) in enumerate(cv.split(trn, y), 1):
    print(f'training model for CV #{i}')
    clf = lgb.LGBMClassifier(**params)
    clf.fit(trn[i_trn], y[i_trn],
            eval_set=[(trn[i_val], y[i_val])],
            eval_metric='multiclass',
            early_stopping_rounds=10)
    
    p_val[i_val, :] = clf.predict_proba(trn[i_val])
    p_tst += clf.predict_proba(tst) / n_fold

training model for CV #1
[1]	valid_0's multi_logloss: 0.970273
Training until validation scores don't improve for 10 rounds
[2]	valid_0's multi_logloss: 0.95501
[3]	valid_0's multi_logloss: 0.940154
[4]	valid_0's multi_logloss: 0.925722
[5]	valid_0's multi_logloss: 0.911652
[6]	valid_0's multi_logloss: 0.897974
[7]	valid_0's multi_logloss: 0.884638
[8]	valid_0's multi_logloss: 0.871616
[9]	valid_0's multi_logloss: 0.858966
[10]	valid_0's multi_logloss: 0.846598
[11]	valid_0's multi_logloss: 0.834526
[12]	valid_0's multi_logloss: 0.822749
[13]	valid_0's multi_logloss: 0.811256
[14]	valid_0's multi_logloss: 0.80004
[15]	valid_0's multi_logloss: 0.789066
[16]	valid_0's multi_logloss: 0.778329
[17]	valid_0's multi_logloss: 0.767822
[18]	valid_0's multi_logloss: 0.757542
[19]	valid_0's multi_logloss: 0.747488
[20]	valid_0's multi_logloss: 0.737674
[21]	valid_0's multi_logloss: 0.728056
[22]	valid_0's multi_logloss: 0.718635
[23]	valid_0's multi_logloss: 0.709421
[24]	valid_0's multi_logloss

[215]	valid_0's multi_logloss: 0.193912
[216]	valid_0's multi_logloss: 0.193422
[217]	valid_0's multi_logloss: 0.192939
[218]	valid_0's multi_logloss: 0.192458
[219]	valid_0's multi_logloss: 0.191984
[220]	valid_0's multi_logloss: 0.191514
[221]	valid_0's multi_logloss: 0.191051
[222]	valid_0's multi_logloss: 0.190595
[223]	valid_0's multi_logloss: 0.190143
[224]	valid_0's multi_logloss: 0.189693
[225]	valid_0's multi_logloss: 0.189252
[226]	valid_0's multi_logloss: 0.188821
[227]	valid_0's multi_logloss: 0.188393
[228]	valid_0's multi_logloss: 0.187971
[229]	valid_0's multi_logloss: 0.187555
[230]	valid_0's multi_logloss: 0.18714
[231]	valid_0's multi_logloss: 0.186732
[232]	valid_0's multi_logloss: 0.186334
[233]	valid_0's multi_logloss: 0.185937
[234]	valid_0's multi_logloss: 0.185546
[235]	valid_0's multi_logloss: 0.185161
[236]	valid_0's multi_logloss: 0.184778
[237]	valid_0's multi_logloss: 0.184402
[238]	valid_0's multi_logloss: 0.18403
[239]	valid_0's multi_logloss: 0.183663
[2

[421]	valid_0's multi_logloss: 0.156471
[422]	valid_0's multi_logloss: 0.156423
[423]	valid_0's multi_logloss: 0.156376
[424]	valid_0's multi_logloss: 0.156333
[425]	valid_0's multi_logloss: 0.156288
[426]	valid_0's multi_logloss: 0.156241
[427]	valid_0's multi_logloss: 0.156197
[428]	valid_0's multi_logloss: 0.156154
[429]	valid_0's multi_logloss: 0.156111
[430]	valid_0's multi_logloss: 0.156071
[431]	valid_0's multi_logloss: 0.156028
[432]	valid_0's multi_logloss: 0.155987
[433]	valid_0's multi_logloss: 0.155942
[434]	valid_0's multi_logloss: 0.155903
[435]	valid_0's multi_logloss: 0.155863
[436]	valid_0's multi_logloss: 0.155821
[437]	valid_0's multi_logloss: 0.15578
[438]	valid_0's multi_logloss: 0.15574
[439]	valid_0's multi_logloss: 0.155699
[440]	valid_0's multi_logloss: 0.155658
[441]	valid_0's multi_logloss: 0.155621
[442]	valid_0's multi_logloss: 0.155582
[443]	valid_0's multi_logloss: 0.155543
[444]	valid_0's multi_logloss: 0.155505
[445]	valid_0's multi_logloss: 0.155465
[4

[634]	valid_0's multi_logloss: 0.152138
[635]	valid_0's multi_logloss: 0.152131
[636]	valid_0's multi_logloss: 0.152125
[637]	valid_0's multi_logloss: 0.152122
[638]	valid_0's multi_logloss: 0.152117
[639]	valid_0's multi_logloss: 0.15211
[640]	valid_0's multi_logloss: 0.152103
[641]	valid_0's multi_logloss: 0.152095
[642]	valid_0's multi_logloss: 0.152086
[643]	valid_0's multi_logloss: 0.152081
[644]	valid_0's multi_logloss: 0.152077
[645]	valid_0's multi_logloss: 0.15207
[646]	valid_0's multi_logloss: 0.152061
[647]	valid_0's multi_logloss: 0.152056
[648]	valid_0's multi_logloss: 0.152052
[649]	valid_0's multi_logloss: 0.152047
[650]	valid_0's multi_logloss: 0.152042
[651]	valid_0's multi_logloss: 0.152033
[652]	valid_0's multi_logloss: 0.152028
[653]	valid_0's multi_logloss: 0.152024
[654]	valid_0's multi_logloss: 0.152019
[655]	valid_0's multi_logloss: 0.152015
[656]	valid_0's multi_logloss: 0.152011
[657]	valid_0's multi_logloss: 0.152008
[658]	valid_0's multi_logloss: 0.152
[659]

[848]	valid_0's multi_logloss: 0.151461
[849]	valid_0's multi_logloss: 0.151458
[850]	valid_0's multi_logloss: 0.15146
[851]	valid_0's multi_logloss: 0.15146
[852]	valid_0's multi_logloss: 0.151457
[853]	valid_0's multi_logloss: 0.151455
[854]	valid_0's multi_logloss: 0.151454
[855]	valid_0's multi_logloss: 0.151451
[856]	valid_0's multi_logloss: 0.151451
[857]	valid_0's multi_logloss: 0.151451
[858]	valid_0's multi_logloss: 0.15145
[859]	valid_0's multi_logloss: 0.151447
[860]	valid_0's multi_logloss: 0.151447
[861]	valid_0's multi_logloss: 0.151445
[862]	valid_0's multi_logloss: 0.151446
[863]	valid_0's multi_logloss: 0.151445
[864]	valid_0's multi_logloss: 0.151446
[865]	valid_0's multi_logloss: 0.151446
[866]	valid_0's multi_logloss: 0.151444
[867]	valid_0's multi_logloss: 0.151443
[868]	valid_0's multi_logloss: 0.151444
[869]	valid_0's multi_logloss: 0.151443
[870]	valid_0's multi_logloss: 0.151443
[871]	valid_0's multi_logloss: 0.151441
[872]	valid_0's multi_logloss: 0.15144
[873

...

[883]	valid_0's multi_logloss: 0.152015
[884]	valid_0's multi_logloss: 0.152012
[885]	valid_0's multi_logloss: 0.152012
[886]	valid_0's multi_logloss: 0.152008
[887]	valid_0's multi_logloss: 0.152007
[888]	valid_0's multi_logloss: 0.152006
[889]	valid_0's multi_logloss: 0.152004
[890]	valid_0's multi_logloss: 0.152003
[891]	valid_0's multi_logloss: 0.152002
[892]	valid_0's multi_logloss: 0.152
[893]	valid_0's multi_logloss: 0.151999
[894]	valid_0's multi_logloss: 0.151996
[895]	valid_0's multi_logloss: 0.151999
[896]	valid_0's multi_logloss: 0.151999
[897]	valid_0's multi_logloss: 0.152
[898]	valid_0's multi_logloss: 0.151997
[899]	valid_0's multi_logloss: 0.151996
[900]	valid_0's multi_logloss: 0.151995
[901]	valid_0's multi_logloss: 0.151991
[902]	valid_0's multi_logloss: 0.151991
[903]	valid_0's multi_logloss: 0.151989
[904]	valid_0's multi_logloss: 0.151991
[905]	valid_0's multi_logloss: 0.151991
[906]	valid_0's multi_logloss: 0.151991
[907]	valid_0's multi_logloss: 0.151989
[908]	

[170]	valid_0's multi_logloss: 0.22399
[171]	valid_0's multi_logloss: 0.223088
[172]	valid_0's multi_logloss: 0.222198
[173]	valid_0's multi_logloss: 0.221322
[174]	valid_0's multi_logloss: 0.22046
[175]	valid_0's multi_logloss: 0.21961
[176]	valid_0's multi_logloss: 0.21877
[177]	valid_0's multi_logloss: 0.217941
[178]	valid_0's multi_logloss: 0.217123
[179]	valid_0's multi_logloss: 0.216314
[180]	valid_0's multi_logloss: 0.215522
[181]	valid_0's multi_logloss: 0.214735
[182]	valid_0's multi_logloss: 0.213958
[183]	valid_0's multi_logloss: 0.21319
[184]	valid_0's multi_logloss: 0.212432
[185]	valid_0's multi_logloss: 0.211686
[186]	valid_0's multi_logloss: 0.210952
[187]	valid_0's multi_logloss: 0.210224
[188]	valid_0's multi_logloss: 0.209512
[189]	valid_0's multi_logloss: 0.208804
[190]	valid_0's multi_logloss: 0.208107
[191]	valid_0's multi_logloss: 0.20742
[192]	valid_0's multi_logloss: 0.206744
[193]	valid_0's multi_logloss: 0.206074
[194]	valid_0's multi_logloss: 0.205414
[195]	

[378]	valid_0's multi_logloss: 0.158286
[379]	valid_0's multi_logloss: 0.158211
[380]	valid_0's multi_logloss: 0.15814
[381]	valid_0's multi_logloss: 0.158068
[382]	valid_0's multi_logloss: 0.157999
[383]	valid_0's multi_logloss: 0.157925
[384]	valid_0's multi_logloss: 0.157858
[385]	valid_0's multi_logloss: 0.157788
[386]	valid_0's multi_logloss: 0.157718
[387]	valid_0's multi_logloss: 0.15765
[388]	valid_0's multi_logloss: 0.157582
[389]	valid_0's multi_logloss: 0.157516
[390]	valid_0's multi_logloss: 0.157451
[391]	valid_0's multi_logloss: 0.157388
[392]	valid_0's multi_logloss: 0.157324
[393]	valid_0's multi_logloss: 0.15726
[394]	valid_0's multi_logloss: 0.157196
[395]	valid_0's multi_logloss: 0.157135
[396]	valid_0's multi_logloss: 0.157072
[397]	valid_0's multi_logloss: 0.157014
[398]	valid_0's multi_logloss: 0.156953
[399]	valid_0's multi_logloss: 0.156894
[400]	valid_0's multi_logloss: 0.156838
[401]	valid_0's multi_logloss: 0.156779
[402]	valid_0's multi_logloss: 0.15672
[403

[593]	valid_0's multi_logloss: 0.15162
[594]	valid_0's multi_logloss: 0.151609
[595]	valid_0's multi_logloss: 0.1516
[596]	valid_0's multi_logloss: 0.151589
[597]	valid_0's multi_logloss: 0.15158
[598]	valid_0's multi_logloss: 0.151572
[599]	valid_0's multi_logloss: 0.151566
[600]	valid_0's multi_logloss: 0.151558
[601]	valid_0's multi_logloss: 0.151548
[602]	valid_0's multi_logloss: 0.151536
[603]	valid_0's multi_logloss: 0.151528
[604]	valid_0's multi_logloss: 0.151519
[605]	valid_0's multi_logloss: 0.151511
[606]	valid_0's multi_logloss: 0.151501
[607]	valid_0's multi_logloss: 0.151493
[608]	valid_0's multi_logloss: 0.151483
[609]	valid_0's multi_logloss: 0.151473
[610]	valid_0's multi_logloss: 0.151465
[611]	valid_0's multi_logloss: 0.151458
[612]	valid_0's multi_logloss: 0.151449
[613]	valid_0's multi_logloss: 0.151439
[614]	valid_0's multi_logloss: 0.15143
[615]	valid_0's multi_logloss: 0.151423
[616]	valid_0's multi_logloss: 0.151415
[617]	valid_0's multi_logloss: 0.151407
[618]

[799]	valid_0's multi_logloss: 0.150635
[800]	valid_0's multi_logloss: 0.150634
[801]	valid_0's multi_logloss: 0.15063
[802]	valid_0's multi_logloss: 0.150627
[803]	valid_0's multi_logloss: 0.150625
[804]	valid_0's multi_logloss: 0.150625
[805]	valid_0's multi_logloss: 0.150621
[806]	valid_0's multi_logloss: 0.150618
[807]	valid_0's multi_logloss: 0.150616
[808]	valid_0's multi_logloss: 0.150615
[809]	valid_0's multi_logloss: 0.150614
[810]	valid_0's multi_logloss: 0.150613
[811]	valid_0's multi_logloss: 0.150612
[812]	valid_0's multi_logloss: 0.150609
[813]	valid_0's multi_logloss: 0.150606
[814]	valid_0's multi_logloss: 0.150605
[815]	valid_0's multi_logloss: 0.150603
[816]	valid_0's multi_logloss: 0.150607
[817]	valid_0's multi_logloss: 0.1506
[818]	valid_0's multi_logloss: 0.150597
[819]	valid_0's multi_logloss: 0.150594
[820]	valid_0's multi_logloss: 0.150591
[821]	valid_0's multi_logloss: 0.150589
[822]	valid_0's multi_logloss: 0.15059
[823]	valid_0's multi_logloss: 0.150589
[824

In [12]:

print(f'{accuracy_score(y, np.argmax(p_val, axis=1)) * 100:.4f}%')

93.6347%


In [13]:
np.savetxt(p_val_file, p_val, fmt='%.6f', delimiter=',')
np.savetxt(p_tst_file, p_tst, fmt='%.6f', delimiter=',')

In [14]:
sub = pd.read_csv(sample_file, index_col=0)
print(sub.shape)
sub.head()

(80000, 1)


,class
id,
320000,0
320001,0
320002,0
320003,0
320004,0


In [15]:

sub[target_col] = np.argmax(p_tst, axis=1)
sub.head()

,class
id,
320000,2
320001,0
320002,2
320003,0
320004,2


In [16]:
sub[target_col].value_counts()

2    40861
0    29996
1     9143
Name: class, dtype: int64

In [17]:
sub.to_csv(sub_file)